In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset.` Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
dataset.info()

In [ ]:
categorical_columns=['sex','smoker','region']
for i in categorical_columns:
  # Convert 'Category' column to categorical type
  dataset[i] = dataset[i].astype('category')
  # Convert categorical data to numerical codes
  dataset[i]=dataset[i].cat.codes

dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

In [ ]:
train_labels=train_dataset.pop('expenses')
test_labels=test_dataset.pop('expenses')

In [ ]:
from sklearn.preprocessing import StandardScaler


# Split datasets into features and labels
x_train = train_dataset.values  # Features for training data
y_train = train_labels.values  # Labels for training data
test_dataset = test_dataset.values  # Features for test data
test_labels = test_labels.values  # Labels for test data




In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(x_train))

#Build the linear regression model
model = Sequential([
    normalizer,
    Dense(16),  # Linear regression layer
    Dense(4),
    Dense(1)
])
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.05),
    loss='mae',
    metrics=['mae', 'mse']
)
model.build()
model.summary()



In [ ]:
#Train the model
model.fit(x_train,
     y_train,
     validation_split=0.2,
     batch_size=32,
     epochs=20)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
